<center>
    <p style="text-align:center">
        <img alt="phoenix logo" src="https://storage.googleapis.com/arize-phoenix-assets/assets/phoenix-logo-light.svg" width="200"/>
        <br>
        <a href="https://arize.com/docs/phoenix/">Docs</a>
        |
        <a href="https://github.com/Arize-ai/phoenix">GitHub</a>
        |
        <a href="https://arize-ai.slack.com/join/shared_invite/zt-2w57bhem8-hq24MB6u7yE_ZF_ilOYSBw#/shared-invite/email">Community</a>
    </p>
</center>
<h1 align="center">Python Quickstart Guide</h1>

This notebook covers the code material from all of the Python Quickstart Guides. Check them out here: 

🔹 [Get Started with Tracing](https://arize.com/docs/phoenix/get-started/get-started-tracing)

🔹 [Get Started with Evals](https://arize.com/docs/phoenix/get-started/get-started-evaluations)

🔹 [Get Started with Prompts](https://arize.com/docs/phoenix/get-started/get-started-prompt-playground) 

🔹 [Get Started with Experiments](https://arize.com/docs/phoenix/get-started/get-started-datasets-and-experiments) 

This notebook will cover creating a multi-agent system using the CrewAI framework, collecting traces, running evals, iterating on the agent, and running experiments. The agent system being built is a Financial Analysis and Research Chatbot, where the inputs are tickers & a focus and the output is a financial report with relevant, recent research. 



### Get Started with Tracing

In [ ]:
%pip install -qqqqq arize-phoenix crewai crewai-tools openinference-instrumentation-crewai openai getpass

In [ ]:
import os
from getpass import getpass

if not (openai_api_key := os.getenv("OPENAI_API_KEY")):
    openai_api_key = getpass("🔑 Enter your OpenAI API key: ")

if not (serper_api_key := os.getenv("SERPER_API_KEY")):
    serper_api_key = getpass("🔑 Enter your Serper API key: ")

if not (phoenix_api_key := os.getenv("PHOENIX_API_KEY")):
    phoenix_api_key = getpass("🔑 Enter your Phoenix API key: ")

if not (phoenix_endpoint := os.getenv("PHOENIX_COLLECTOR_ENDPOINT")):
    phoenix_endpoint = getpass("🔑 Enter your Phoenix Endpoint: ")

os.environ["PHOENIX_API_KEY"] = phoenix_api_key
os.environ["PHOENIX_COLLECTOR_ENDPOINT"] = phoenix_endpoint
os.environ["SERPER_API_KEY"] = serper_api_key
os.environ["OPENAI_API_KEY"] = openai_api_key

In [ ]:
from phoenix.otel import register

tracer_provider = register(project_name="crewai-tracing-quickstart", auto_instrument=True)

In [ ]:
from crewai import Agent, Crew, Process, Task
from crewai_tools import SerperDevTool

search_tool = SerperDevTool()

researcher = Agent(
    role="Financial Research Analyst",
    goal="Gather up-to-date financial data, trends, and news for the target companies or markets",
    backstory="""
        You are a Senior Financial Research Analyst.
    """,
    verbose=True,
    allow_delegation=False,
    max_iter=2,
    tools=[search_tool],
)

writer = Agent(
    role="Financial Report Writer",
    goal="Compile and summarize financial research into clear, actionable insights",
    backstory="""
        You are an experienced financial content writer.
    """,
    verbose=True,
    allow_delegation=True,
    max_iter=1,
)

task1 = Task(
    description="""
        Research: {tickers}
        Focus on: {focus}
    """,
    expected_output="Detailed financial research summary with web search findings",
    agent=researcher,
)

task2 = Task(
    description="Write a report based on the research above.",
    expected_output="A polished financial analysis report",
    agent=writer,
)

crew = Crew(
    agents=[researcher, writer],
    tasks=[task1, task2],
    verbose=1,
    process=Process.sequential,
)

In [ ]:
user_inputs = {"tickers": "TSLA", "focus": "financial analysis and market outlook"}

result = crew.kickoff(inputs=user_inputs)

### Get Started with Evaluations

In [ ]:
test_queries = [
    {"tickers": "AAPL", "focus": "financial analysis and market outlook"},
    {"tickers": "NVDA", "focus": "valuation metrics and growth prospects"},
    {"tickers": "AMZN", "focus": "profitability and market share"},
    {"tickers": "AAPL, MSFT", "focus": "comparative financial analysis"},
    {"tickers": "META, SNAP, PINS", "focus": "social media sector trends"},
    {"tickers": "RIVN", "focus": "financial health and viability"},
    {"tickers": "SNOW", "focus": "revenue growth trajectory"},
    {"tickers": "KO", "focus": "dividend yield and stability"},
    {"tickers": "META", "focus": "latest developments and stock performance"},
    {
        "tickers": "AAPL, MSFT, GOOGL, AMZN, META",
        "focus": "big tech comparison and market outlook",
    },
    {"tickers": "AMC", "focus": "financial analysis and market sentiment"},
]

In [ ]:
for query in test_queries:
    crew.kickoff(inputs=query)

In [ ]:
financial_completeness_template = """
You are evaluating whether a financial research report correctly completes ALL parts of the user's task with COMPREHENSIVE coverage.

User input: {attributes.input.value}

Generated report:
{attributes.output.value}

To be marked as "complete", the report MUST meet ALL of these strict requirements:

1. TICKER COVERAGE (MANDATORY):
   - Cover ALL companies/tickers mentioned in the input
   - If multiple tickers are listed, EACH must have dedicated analysis (not just mentioned in passing)
   - For multiple tickers, the report must provide COMPARATIVE analysis when relevant

2. FOCUS AREA COVERAGE (MANDATORY):
   - Address ALL focus areas mentioned in the input
   - If the focus mentions multiple topics (e.g., "earnings and outlook"), BOTH must be thoroughly addressed
   - Each focus area must have substantial content, not just a brief mention

3. FINANCIAL DATA REQUIREMENTS (MANDATORY):
   - For EACH ticker, the report must include:
     * Current/recent stock price or performance data
     * At least 2 key financial ratios (P/E, P/B, debt-to-equity, ROE, etc.)
     * Revenue or earnings information
     * Recent news or developments (within last 6 months)
   - If focus mentions specific metrics (e.g., "P/E ratio"), those MUST be explicitly provided

4. DEPTH REQUIREMENT (MANDATORY):
   - Each ticker must have at least 3-4 sentences of dedicated analysis
   - Generic statements without specific data do NOT count
   - The report must demonstrate thorough research, not superficial coverage

5. COMPARISON REQUIREMENT (if multiple tickers):
   - If 2+ tickers are requested, the report MUST include direct comparisons
   - Comparisons should cover multiple key metrics side-by-side
   - Generic statements like "both companies are good" do NOT satisfy this requirement
   - Must explicitly state which company performs better/worse on specific metrics

The report is "incomplete" if it fails ANY of the above requirements, including:
- Missing any ticker or only mentioning it briefly
- Failing to address any focus area or only addressing it superficially
- Missing required financial data for any ticker
- Providing generic analysis without specific metrics or data
- Failing to provide comparisons when multiple tickers are requested
- Not meeting the depth requirement for any ticker

Respond with ONLY one word: "complete" or "incomplete"
Then provide a detailed explanation of which specific requirements were met or failed.
"""

In [ ]:
from phoenix.client import Client

px_client = Client()
df = px_client.spans.get_spans_dataframe(project_name="crewai-tracing-quickstart")
parent_spans = df[df["span_kind"] == "CHAIN"]

In [ ]:
from phoenix.evals import LLM

llm = LLM(model="gpt-4o", provider="openai")

In [ ]:
from phoenix.evals import create_classifier

completness_evaluator = create_classifier(
    name="completeness",
    prompt_template=financial_completeness_template,
    llm=llm,
    choices={"complete": 1.0, "incomplete": 0.0},
)

In [ ]:
from phoenix.evals import evaluate_dataframe
from phoenix.trace import suppress_tracing

with suppress_tracing():
    results_df = evaluate_dataframe(dataframe=parent_spans, evaluators=[completness_evaluator])

In [ ]:
from phoenix.evals.utils import to_annotation_dataframe

evaluations = to_annotation_dataframe(dataframe=results_df)

Client().spans.log_span_annotations_dataframe(dataframe=evaluations)

### Get Started with Experiments

In [ ]:
researcher = Agent(
    role="Financial Research Analyst",
    goal="""Gather up-to-date financial data, trends, and news for the target companies/markets.
        Make sure to include more than 1 financial ratios (such as P/E or P/B), news from the last 6 months, and current stock price or performance data.""",
    backstory="""
            You are a Senior Financial Research Analyst.
        """,
    verbose=True,
    allow_delegation=False,
    max_iter=2,
    tools=[search_tool],
)
writer = Agent(
    role="Financial Report Writer",
    goal="Compile and summarize financial research into clear, actionable insights. If there are multiple tickers, make sure to include a dedicated comparison section.",
    backstory="""
        You are an experienced financial content writer.
    """,
    verbose=True,
    allow_delegation=True,
    max_iter=1,
)
updated_crew = Crew(
    agents=[researcher, writer],
    tasks=[task1, task2],
    verbose=1,
    process=Process.sequential,
)

In [ ]:
def my_task(example):
    result = updated_crew.kickoff(inputs=example.input)
    return result

In [ ]:
from phoenix.evals import ClassificationEvaluator

completeness_evaluator = ClassificationEvaluator(
    name="completeness",
    prompt_template=financial_completeness_template,
    llm=llm,
    choices={"complete": 1.0, "incomplete": 0.0},
)


def completeness(input, output):
    results = completeness_evaluator.evaluate(
        {"attributes.input.value": input, "attributes.output.value": output}
    )
    return results[0].label


evaluators = [completeness]

In [ ]:
dataset = Client().datasets.get_dataset(dataset="python quickstart fails")

In [ ]:
from phoenix.experiments import run_experiment

experiment = run_experiment(dataset, my_task, evaluators)